In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
import sys
import time


In [2]:
def binarioPro(ul,u,ur,l,pix,r,dl,d,dr):
    res = np.zeros(pix.shape)
    vecinos = np.array([ul,u,ur,r,dr,d,dl,l])
    for i in range(len(vecinos)):
        res = res + np.power(2,i)*(vecinos[i] >=pix)
    return res

In [4]:
def extraccionCaracteristicas(imagen, N,M,nBloques,mBloques): 

    imgAmp = cv2.copyMakeBorder(imagen,1,1,1,1,cv2.BORDER_REPLICATE)

    imagenCambio = binarioPro(imgAmp[0:-2,0:-2],imgAmp[0:-2,1:-1],imgAmp[0:-2,2:],imgAmp[1:-1,0:-2],imgAmp[1:-1,1:-1],imgAmp[1:-1,2:],imgAmp[2:,0:-2],imgAmp[2:,1:-1],imgAmp[2:,2:])
    
    listaH = np.zeros((256 * nBloques * mBloques,3))
    
    for i in range(nBloques):
        for j in range(mBloques):
            listaH[(j*nBloques + i)*256 : (j*nBloques + i + 1)*256,0] = cv2.calcHist(np.uint8(imagenCambio[i*N:(i+1)*N, j*M:(j+1)*M]),[0], None , [256] , (0,256) , False).ravel()
            listaH[(j*nBloques + i)*256 : (j*nBloques + i + 1)*256,1] = cv2.calcHist(np.uint8(imagenCambio[i*N:(i+1)*N, j*M:(j+1)*M]),[1], None , [256] , (0,256) , False).ravel()
            listaH[(j*nBloques + i)*256 : (j*nBloques + i + 1)*256,2] = cv2.calcHist(np.uint8(imagenCambio[i*N:(i+1)*N, j*M:(j+1)*M]),[2], None , [256] , (0,256) , False).ravel()
    return listaH

In [5]:
def im_to_col(imagen, m, n):
    filas, columnas = imagen.shape
    a = m//2
    b = n//2
    imagen2 = np.zeros((m*n, filas*columnas))
    imagen_amp = cv2.copyMakeBorder(imagen, a, a, b, b, cv2.BORDER_REPLICATE)
    aux = 0
    for i in range(a, filas+a):
        for j in range(b, columnas+b):
            imagen2[:, aux] = imagen_amp[i-a:i+a+1, j-b:j+b+1].flatten()
            aux += 1
    return imagen2

In [14]:
def feature_extraction(image, N, R):
    # Obtenemos las dimensiones de la imagen
    height, width = image.shape
    
    # Calculamos el número de bloques en filas y columnas
    num_blocks_rows = height // N
    num_blocks_cols = width // N


    imCol = im_to_col(image, R, R).T

    imagenBinaria = np.zeros(len(imCol))

    for i in range(len(imCol)):   
        imagenBinaria[i] = np.sum(np.multiply(np.where(np.greater_equal(imCol[i],imCol[i,(R*R)//2]), 1, 0),np.array([[128, 64, 32 , 1, 0, 16 , 2, 4, 8]]))  )

    imagenBinaria = imagenBinaria.reshape(image.shape)

    # Dividimos la imagen en bloques de N filas
    blocks = np.vsplit(imagenBinaria, num_blocks_rows)
   
    histograms = []

    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques de N columnas
        cells = np.hsplit(block, num_blocks_cols)

        # Iteramos sobre las celdas
        for j, celda in enumerate(cells):           

            # Calculamos el histograma del bloque            
            
          
            histograms.append(cv2.calcHist([np.uint8(celda)], [0], None, [256], [0, 256]))

    return np.array(histograms).flatten()


In [12]:
def feature_extractionOptimo(image, N, R):
    # Obtenemos las dimensiones de la imagen
    height, width = image.shape
    
    # Calculamos el número de bloques en filas y columnas
    num_blocks_rows = height // N
    num_blocks_cols = width // N


    imgAmp = cv2.copyMakeBorder(image,1,1,1,1,cv2.BORDER_REPLICATE)

    imagenBinaria = binarioPro(imgAmp[0:-2,0:-2],imgAmp[0:-2,1:-1],imgAmp[0:-2,2:],imgAmp[1:-1,0:-2],imgAmp[1:-1,1:-1],imgAmp[1:-1,2:],imgAmp[2:,0:-2],imgAmp[2:,1:-1],imgAmp[2:,2:])


    # Dividimos la imagen en bloques de N filas
    blocks = np.vsplit(image, num_blocks_rows)
   
    histograms = np.zeros((num_blocks_rows*num_blocks_cols, 256,1))

    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques de N columnas
        cells = np.hsplit(block, num_blocks_cols)
    
        # Iteramos sobre las celdas
        for j, celda in enumerate(cells):        
            
            # Calculamos el histograma del bloque
            histograms[j] = cv2.calcHist([np.uint8(celda)], [0], None, [256], [0, 256])

    return histograms.flatten()

In [15]:
#MALARDO

datasets = ["cat_dog_100"]

N = 16
R = 3

HEIGHT =  384
WIDTH = 528

num_blocks_rows = HEIGHT // N
num_blocks_cols = WIDTH // N

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path , 0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)            
            
            
            features = feature_extraction(resized_image, N, R);       

            X_train.append(features.flatten())
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path,0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
                
            features = feature_extraction(resized_image, N, R);   

            
            X_test.append(features.flatten())
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
        print("")
    print("")
    
    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train, np.uint8)
    y_train = np.array(y_train)
    X_test = np.array(X_test, np.uint8)
    y_test = np.array(y_test)


    print("SHAPE Xtrin",X_train.shape)
    print("SHAPE y_train",y_train.shape)
    print("SHAPE X_test",X_test.shape)
    print("SHAPE y_test",y_test.shape)
    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))


    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")
    


dataset:  cat_dog_100
Procesamos las imágenes de entrenamiento
Procesando label:  cat
 Procesando Xtrain ... (100 %)
Procesando label:  dog
 Procesando Xtrain ... (100 %)
Procesamos las imágenes de prueba
Procesando label:  cat
 Procesando Xtest ... (100 %)
Procesando label:  dog
 Procesando Xtest ... (100 %)

SHAPE Xtrin (200, 202752)
SHAPE y_train (200,)
SHAPE X_test (40, 202752)
SHAPE y_test (40,)
Precisión: 95.00%
Tiempo transcurrido: 14 minutos 30.17 segundos


In [17]:
#Optimo

datasets = ["cat_dog_100"]

N = 16
R = 3

HEIGHT =  384
WIDTH = 528

num_blocks_rows = HEIGHT // N
num_blocks_cols = WIDTH // N

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path , 0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)            
            
            
            features = feature_extractionOptimo(resized_image, N, R);       

            X_train.append(features.flatten())
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path,0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
                
            features = feature_extractionOptimo(resized_image, N, R);   

            
            X_test.append(features.flatten())
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
        print("")
    print("")
    
    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train, np.uint8)
    y_train = np.array(y_train)
    X_test = np.array(X_test, np.uint8)
    y_test = np.array(y_test)


    print("SHAPE Xtrin",X_train.shape)
    print("SHAPE y_train",y_train.shape)
    print("SHAPE X_test",X_test.shape)
    print("SHAPE y_test",y_test.shape)
    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))


    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")
    

dataset:  cat_dog_100
Procesamos las imágenes de entrenamiento
Procesando label:  cat
 Procesando Xtrain ... (100 %)
Procesando label:  dog
 Procesando Xtrain ... (100 %)
Procesamos las imágenes de prueba
Procesando label:  cat
 Procesando Xtest ... (100 %)
Procesando label:  dog
 Procesando Xtest ... (100 %)

SHAPE Xtrin (200, 202752)
SHAPE y_train (200,)
SHAPE X_test (40, 202752)
SHAPE y_test (40,)
Precisión: 60.00%
Tiempo transcurrido: 0 minutos 16.04 segundos


In [41]:
image = cv2.imread("test.jpg",0)

startTimer = time.perf_counter()

iamgenBin = feature_extraction(image, 16, 3)

# Detenemos el contador de tiempo
endTimer = time.perf_counter()

# Calculamos el tiempo transcurrido
elapsed = endTimer - startTimer
# Obtenemos el tiempo transcurrido en minutos y segundos
minutes, seconds = divmod(elapsed, 60)

# Mostramos el tiempo transcurrido en minutos y segundos
print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.6f} segundos")

startTimer = time.perf_counter()

imgAmp = cv2.copyMakeBorder(image,1,1,1,1,cv2.BORDER_REPLICATE)

imagenCambio = binarioPro(imgAmp[0:-2,0:-2],imgAmp[0:-2,1:-1],imgAmp[0:-2,2:],imgAmp[1:-1,0:-2],imgAmp[1:-1,1:-1],imgAmp[1:-1,2:],imgAmp[2:,0:-2],imgAmp[2:,1:-1],imgAmp[2:,2:])

# Detenemos el contador de tiempo
endTimer = time.perf_counter()

# Calculamos el tiempo transcurrido
elapsed = endTimer - startTimer
# Obtenemos el tiempo transcurrido en minutos y segundos
minutes, seconds = divmod(elapsed, 60)

# Mostramos el tiempo transcurrido en minutos y segundos
print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.6f} segundos")

# Mostrar la imagen en una ventana llamada "Image"
cv2.imshow("Image", iamgenBin)

cv2.imshow("Image2", imagenCambio)
# Mantener la ventana abierta hasta que se pulse una tecla
cv2.waitKey(0)

# Cerrar la ventana
cv2.destroyAllWindows()

Tiempo transcurrido: 0 minutos 0.384787 segundos
Tiempo transcurrido: 0 minutos 0.000818 segundos
